In [1]:
import complexnn
import tensorflow as tf
import numpy as np
from keras.models import Sequential
from keras.layers import Layer
import keras
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)

    except RuntimeError as e:
        print(e)
np.random.seed(1)
x_train = np.random.rand(61,64,64,4)
y_train = np.random.rand(61,64,64,8)

Using TensorFlow backend.


## Amplitude maxout

In [2]:
@tf.function
def AMU(x, num_units=4, axis=None):
    shape = x.get_shape().as_list()
    if axis is None:
        axis = -1
        shape[0] = -1
    num_channels = shape[axis]//2
    shape[axis] = num_units
    if num_channels % num_units:
        raise ValueError(f'nb of real/imaginary channel {num_channels} is not a multiple of {num_units}')
    exp_shape = shape + [num_channels//num_units]
    real_part = tf.reshape(x[:,:,:,:num_channels],exp_shape)
    imag_part = tf.reshape(x[:,:,:,num_channels:],exp_shape)
    modulus = x[:,:,:,:num_channels]**2 + x[:,:,:,num_channels:]**2
    exp_modulus = tf.reshape(modulus,exp_shape)
    cond = tf.where(exp_modulus==tf.expand_dims(tf.reduce_max(exp_modulus, axis=-1),-1),True,False)

    return tf.concat([tf.reshape(real_part[cond],shape),tf.reshape(imag_part[cond],shape)],axis=-1)

In [3]:
@tf.function
def maxout(x, num_units=2, axis=None):
    shape = x.get_shape().as_list()
    if axis is None:
        axis = -1
    if shape[axis]%num_units:
        raise ValueError(f'nb of fetures {shape[-2]} is not a multiple of {num_units}')
    num_channel = shape[axis]    
    shape[axis] = num_units
    shape[0] = -1
    shape = shape + [num_channel//num_units]
    return tf.reduce_max(tf.reshape(x,shape), axis=-1, keepdims=False)


In [2]:
class AmplitudeMaxout(Layer):
    def __init__(self, num_units, name='AMU',**kwargs):
        super().__init__(**kwargs)
        self.__name__ = name
        self.num_units = num_units
    @tf.function
    def call(self, x, axis=None):
        shape = x.get_shape().as_list()
        if axis is None:
            axis = -1
            shape[0] = -1
        num_channels = shape[axis]//2
        shape[axis] = self.num_units
        if num_channels % self.num_units:
            raise ValueError(f'nb of real/imaginary channel {num_channels} is not a multiple of {self.num_units}')
        exp_shape = shape + [num_channels//self.num_units]
        real_part = tf.reshape(x[:,:,:,:num_channels],exp_shape)
        imag_part = tf.reshape(x[:,:,:,num_channels:],exp_shape)
        modulus = x[:,:,:,:num_channels]**2 + x[:,:,:,num_channels:]**2
        exp_modulus = tf.reshape(modulus,exp_shape)
        cond = tf.where(exp_modulus==tf.expand_dims(tf.reduce_max(exp_modulus, axis=-1),-1),True,False)
        return tf.concat([tf.reshape(real_part[cond],shape),tf.reshape(imag_part[cond],shape)],axis=-1)
    def compute_output_shape(self, input_shape):
        shape = list(input_shape)
        shape[-1] = 2*self.num_units
        return tuple(shape)
    def get_config(self):
        base_config = super().get_config()
        config = {'units': self.num_units}
        return dict(list(base_config.items()) + list(config.items()))

In [334]:
def decorator_AMU(func):
    def run_AMU(y):
        # 再透過first讀取y值也就是2,3*2+執行func獲得的值
        value = func
        return 3*y + value
    return run_AMU
def AMU_here(x):
    # decorator_AMU讀取AMU_here(3)然後獲得first=run_AMU
    return 5*x
first = decorator_AMU(AMU_here(3))
first(2)

21

In [335]:
def decorator_AMU1(func):
    def run_AMU(*args):
        value = func(*args)
        print('now run')
        return 5*value
    return run_AMU
@decorator_AMU1
def AMU_direct_call(x):
    print('now dir')
    return 5*x
AMU_direct_call(2)

now dir
now run


50

# Activation function

In [2]:
def TestAMU(num_units):
    @tf.function
    def return_value(x,axis=None):
        shape = x.get_shape().as_list()
        if axis is None:
            axis = -1
            shape[0] = -1
        num_channels = shape[axis]//2
        shape[axis] = num_units
        if num_channels % num_units:
            raise ValueError(f'nb of real/imaginary channel {num_channels} is not a multiple of {num_units}')
        exp_shape = shape + [num_channels//num_units]
        real_part = tf.reshape(x[:,:,:,:num_channels],exp_shape)
        imag_part = tf.reshape(x[:,:,:,num_channels:],exp_shape)
        modulus = x[:,:,:,:num_channels]**2 + x[:,:,:,num_channels:]**2
        exp_modulus = tf.reshape(modulus,exp_shape)
        cond = tf.where(exp_modulus==tf.expand_dims(tf.reduce_max(exp_modulus, axis=-1),-1),True,False)
        return tf.concat([tf.reshape(real_part[cond],shape),tf.reshape(imag_part[cond],shape)],axis=-1)
    return return_value
@tf.function
def CReLU(x):
    shape = x.get_shape().as_list()
    num_channels = shape[-1]//2
    real_part = tf.nn.relu(x[:,:,:,:num_channels])
    imag_part = tf.nn.relu(x[:,:,:,num_channels:])
    return tf.concat([real_part,imag_part],axis=-1)

@tf.function
def zReLU(x):
    shape = x.get_shape().as_list()
    num_channels = shape[-1]//2
    real_part = x[:,:,:,:num_channels]
    imag_part = x[:,:,:,num_channels:]
    cond = (real_part >= 0) & (imag_part >= 0)
    real_part = tf.where(cond,real_part,0)
    imag_part = tf.where(cond,imag_part,0)
    return tf.concat([real_part, imag_part],axis=-1)
# class modReLU:
#     def __init__(self):
#         self.b = tf.Variable(-1., trainable=True)
#     @tf.function 
#     def call(self, x):
#         shape = x.get_shape().as_list()
#         num_channels = shape[-1]//2
#         real_part = x[:,:,:,:num_channels]
#         imag_part = x[:,:,:,num_channels:]
#         modulus = tf.sqrt(real_part**2 + imag_part**2)
#         phase = tf.atan2(imag_part,real_part)
#         cond = (tf.add(modulus, self.b) >= 0)
#         modulus = tf.where(cond, modulus, 0)
#         real_part = modulus*tf.cos(phase)
#         imag_part = modulus*tf.sin(phase)
#         return tf.concat([real_part,imag_part], axis=-1)
def modReLU(x):
    b = tf.Variable(-4., trainable=True)
    @tf.function 
    def return_value():
        shape = x.get_shape().as_list()
        num_channels = shape[-1]//2
        real_part = x[:,:,:,:num_channels]
        imag_part = x[:,:,:,num_channels:]
        modulus = tf.sqrt(real_part**2 + imag_part**2)
        phase = tf.atan2(imag_part,real_part)
        cond = (tf.add(modulus, b) >= 0)
        modulus = tf.where(cond, modulus, 0)
        real_part = modulus*tf.cos(phase)
        imag_part = modulus*tf.sin(phase)
        return tf.concat([real_part,imag_part], axis=-1)
    return return_value()

In [133]:
x = tf.Variable([[1.,-2.,3.,-5.3,0.],[2.,-3.,-3.,1.5,0.]])
x = tf.reshape(tf.transpose(x),[1,1,5,2])
print(x)
print('crelu',CReLU(x))
print('zrelu',zReLU(x))
print('modRelu',modReLU(x))

tf.Tensor(
[[[[ 1.   2. ]
   [-2.  -3. ]
   [ 3.  -3. ]
   [-5.3  1.5]
   [ 0.   0. ]]]], shape=(1, 1, 5, 2), dtype=float32)
crelu tf.Tensor(
[[[[1.  2. ]
   [0.  0. ]
   [3.  0. ]
   [0.  1.5]
   [0.  0. ]]]], shape=(1, 1, 5, 2), dtype=float32)
zrelu tf.Tensor(
[[[[1. 2.]
   [0. 0.]
   [0. 0.]
   [0. 0.]
   [0. 0.]]]], shape=(1, 1, 5, 2), dtype=float32)
modRelu tf.Tensor(
[[[[ 0.         0.       ]
   [-0.        -0.       ]
   [ 2.9999998 -2.9999998]
   [-5.3        1.4999998]
   [ 0.         0.       ]]]], shape=(1, 1, 5, 2), dtype=float32)


In [178]:
'''
受限於keras，無法處理孤兒
'''
class AmplitudeMaxout_piece(Layer):
    def __init__(self, num_pieces, name='AMU',**kwargs):
        super().__init__(**kwargs)
        self.__name__ = name
        self.num_pieces = num_pieces    
  
    @tf.function
    def call(self, x, axis=None):
        shape = x.get_shape().as_list()   
        if axis is None:
            axis = -1
            shape[0] = -1
        # nb of Re/Im channels
        num_channels = shape[axis]//2
        # if # of Re != Im channels
        if shape[axis]%2:
            raise ValueError(f'nb of real/imaginary channel are inequivalent')
        # Re/Im channel 每num_piece比較一次，因此最後會有num_units個channel    
        self.num_units = num_channels//self.num_pieces
        # 增加一個dim，最後一個dim為比較大小用，最後會被壓縮，壓縮至倒數第二個channel，因此最後會有num_units個channel
        shape[axis] = self.num_units
        exp_shape = shape + [self.num_pieces]
        # 若輸入的Re/Im channel數量無法被num_pieces整除，需分開處理整除以及剩下的部分
        self.num_rest_piece = num_channels%self.num_pieces
        if self.num_rest_piece:
            # 將整除的部分取出
            real_part = tf.reshape(x[:,:,:,:num_channels-self.num_rest_piece],exp_shape)
            imag_part = tf.reshape(x[:,:,:,num_channels:-self.num_rest_piece],exp_shape)
            # 將整除的部分找出AMU
            real_part, imag_part = self.return_AMU(real_part, imag_part, exp_shape)
            # -----------------------------------------------------------
            # 處理剩下的部分，倒數第二個dim必定是1，最後一個為剩下的piece數量
            rest_exp_shape = shape[0:3] + [1,self.num_rest_piece]
            # 將剩餘的部分取出
            rest_real_part = tf.reshape(x[:,:,:,num_channels-self.num_rest_piece:num_channels], rest_exp_shape)
            rest_imag_part = tf.reshape(x[:,:,:,-self.num_rest_piece:], rest_exp_shape)
            rest_real_part, rest_imag_part = self.return_AMU(rest_real_part, rest_imag_part, rest_exp_shape)
            real = tf.concat([real_part,rest_real_part], axis=-1)
            imag = tf.concat([imag_part,rest_imag_part], axis=-1)           
            return tf.concat([real,imag],axis=-1)         
        else:
            print(exp_shape)
            real_part = tf.reshape(x[:,:,:,:num_channels],exp_shape)
            imag_part = tf.reshape(x[:,:,:,num_channels:],exp_shape)
            real_part, imag_part = self.return_AMU(real_part, imag_part, exp_shape)
            print(tf.shape(real_part))
            return tf.concat([real_part,imag_part],axis=-1)

    def compute_output_shape(self, input_shape):
        shape = list(input_shape)
        if self.num_rest_piece:
            shape[-1] = 2*self.num_units + 1
        else:
            shape[-1] = 2*self.num_units
        print(shape)
        return tuple(shape)
    
    def get_config(self):
        base_config = super().get_config()
        config = {'units': self.num_units,
                  'pieces': self.num_pieces}
        return dict(list(base_config.items()) + list(config.items()))
    @tf.function
    def return_AMU(self, real_part, imag_part ,expand_shape):
        # 計算複數絕對值長度
        modulus = real_part**2 + imag_part**2
        # 將最後兩個dim重組成(num_units,num_pieces)
        expand_modulus = tf.reshape(modulus, expand_shape)
        # 找出最大值後重組成原本的dim再根據位置判斷哪個實部虛部造成該最大值
#         cond = tf.where(expand_modulus==tf.expand_dims(tf.reduce_max(expand_modulus, axis=-1),-1),True,False)
    
        cond = tf.equal(expand_modulus,tf.reduce_max(expand_modulus,axis=-1,keepdims=True))
        real_part = tf.reduce_max(real_part*tf.cast(cond,dtype=tf.float32),axis=-1)
        imag_part = tf.reduce_max(imag_part*tf.cast(cond,dtype=tf.float32),axis=-1)
        
        # 過濾出實部虛部，須注意使用[]過濾出的值會是1D，必須再重組
#         real_part = tf.reshape(real_part[cond],expand_shape[:-1])
#         imag_part = tf.reshape(imag_part[cond],expand_shape[:-1])
        
        return real_part, imag_part
# AmplitudeMaxout_piece(4)(tf.random.normal([32,5,5,32],dtype=tf.float32))

In [41]:
class InceptLayer(Layer):
    def __init__(self,
                 filter1,
                 filter2,
                 filter3,
                 filter4,
                 kernel1,
                 kernel2,
                 kernel3,
                 kernel4,
                 num_units=4,
                 name='InceptionLayer',**kwargs):
        super().__init__(**kwargs)
        assert (filter1%num_units)+(filter2%num_units)+(filter3%num_units)+(filter4%num_units) == 0
        self.filter1 = filter1
        self.filter2 = filter2
        self.filter3 = filter3
        self.filter4 = filter4
        self.kernel1 = kernel1
        self.kernel2 = kernel2
        self.kernel3 = kernel3
        self.kernel4 = kernel4
        self.num_units = num_units

    def build(self, x):
        self.conv1 = complexnn.conv.ComplexConv2D(self.filter1,self.kernel1,activation=AmplitudeMaxout(self.num_units),padding='same')
        self.conv2 = complexnn.conv.ComplexConv2D(self.filter2,self.kernel2,activation=AmplitudeMaxout(self.num_units),padding='same')
        self.conv3 = complexnn.conv.ComplexConv2D(self.filter3,self.kernel3,activation=AmplitudeMaxout(self.num_units),padding='same')
        self.conv4 = complexnn.conv.ComplexConv2D(self.filter4,self.kernel4,activation=AmplitudeMaxout(self.num_units),padding='same')
    def call(self, x):
        real1, imag1 = self.seperate(self.conv1(x))
        real2, imag2 = self.seperate(self.conv2(x))
        real3, imag3 = self.seperate(self.conv3(x))
        real4, imag4 = self.seperate(self.conv4(x))
        return tf.concat([real1,real2,real3,real4,imag1,imag2,imag3,imag4], axis=-1)
    def seperate(self, x):
        shape = x.get_shape().as_list()
        dim = shape[-1]//2
        realpart = x[:,:,:,:dim]
        imagpart = x[:,:,:,dim:]
        return realpart, imagpart
#InceptLayer(filter1=8, filter2=8, filter3=8, filter4=8, kernel1=(15,11), kernel2=(17,13), kernel3=(19,15), kernel4=(21,17))     

# Inception layer

In [3]:
class InceptLayer(Layer):
    def __init__(self,
                 filter1,
                 filter2,
                 filter3,
                 filter4,
                 kernel1,
                 kernel2,
                 kernel3,
                 kernel4,
                 num_units=4,
                 name='InceptionLayer',**kwargs):
        super().__init__(**kwargs)
        self.filter1 = filter1
        self.filter2 = filter2
        self.filter3 = filter3
        self.filter4 = filter4
        self.kernel1 = kernel1
        self.kernel2 = kernel2
        self.kernel3 = kernel3
        self.kernel4 = kernel4
        self.num_units = num_units
    def build(self, input_shape):
        print('inputshape',input_shape)
        self.conv1 = complexnn.conv.ComplexConv2D(self.filter1,self.kernel1,padding='same')
        self.conv2 = complexnn.conv.ComplexConv2D(self.filter2,self.kernel2,padding='same')
        self.conv3 = complexnn.conv.ComplexConv2D(self.filter3,self.kernel3,padding='same')
        self.conv4 = complexnn.conv.ComplexConv2D(self.filter4,self.kernel4,padding='same')
        self.act = AmplitudeMaxout_inclued(self.num_units)
    def call(self, x):
        conv_output1 = self.conv1(x)
        real1, imag1 = self.seperate(self.act(conv_output1))
        print('real1',real1.shape,'image1',imag1.shape)
        conv_output2 = self.conv2(x)
        real2, imag2 = self.seperate(self.act(conv_output2))
        print('real2',real2.shape,'image2',imag2.shape)
        conv_output3 = self.conv3(x)
        real3, imag3 = self.seperate(self.act(conv_output3))
        print('real3',real3.shape,'image3',imag3.shape)
        conv_output4 = self.conv4(x)
        real4, imag4 = self.seperate(self.act(conv_output4))
        print('real4',real4.shape,'image4',imag4.shape)
        print('what',tf.concat([real1,real2,real3,real4,imag1,imag2,imag3,imag4], axis=-1).shape)
        return tf.concat([real1,real2,real3,real4,imag1,imag2,imag3,imag4], axis=-1)
    def compute_output_shape(self, input_shape):
        last_dim1 = self.compute_shape(self.filter1,self.num_units,self.filter1%self.num_units)
        print('last_dim1',last_dim1)
        last_dim2 = self.compute_shape(self.filter2,self.num_units,self.filter2%self.num_units)
        last_dim3 = self.compute_shape(self.filter3,self.num_units,self.filter3%self.num_units)
        last_dim4 = self.compute_shape(self.filter4,self.num_units,self.filter4%self.num_units)
        shape = list(input_shape)
        shape[-1] = last_dim1 + last_dim2 + last_dim3 + last_dim4
        print('computeshape',shape)
        return tuple(shape)
    
    def get_config(self):
        pass
    
    def seperate(self, x):
        shape = x.get_shape().as_list()
        dim = shape[-1]//2
        realpart = x[:,:,:,:dim]
        imagpart = x[:,:,:,dim:]
        return realpart, imagpart
    def compute_shape(self,filters,units,flag):
        if flag:
            return 2*(filters//units + 1)
        else:
            return 2*(filters//units)

# Loss function

In [4]:
@tf.function
def ComplexRMS(y_true, y_pred):
    shape = y_pred.get_shape().as_list()
    if shape[-1]%2:
        raise ValueError(f"nb of imaginary part isn't equal to that of real part")
    num_channels = shape[-1]//2
    n_points = 1
    for n in shape[1:]:
        n_points = n_points*n
    n_points = tf.cast(n_points,dtype=tf.float32)/2
    real_pdt = y_pred[:,:,:,num_channels:]
    imag_pdt = y_pred[:,:,:,:num_channels]
    real_true = y_true[:,:,:,num_channels:]
    imag_true = y_true[:,:,:,:num_channels]
    return tf.sqrt(tf.reduce_sum((real_pdt-real_true)**2+(imag_pdt-imag_true)**2)/n_points)
@tf.function
def ComplexMSE(y_true, y_pred):
    shape = y_pred.get_shape().as_list()
    if shape[-1]%2:
        raise ValueError(f"nb of imaginary part isn't equal to that of real part")
    num_channels = shape[-1]//2
    n_points = 1
    for n in shape[1:]:
        n_points = n_points*n
    n_points = tf.cast(n_points,dtype=tf.float32)/2
    real_pdt = y_pred[:,:,:,num_channels:]
    imag_pdt = y_pred[:,:,:,:num_channels]
    real_true = y_true[:,:,:,num_channels:]
    imag_true = y_true[:,:,:,:num_channels]
    return tf.reduce_sum(((real_pdt-real_true)**2+(imag_pdt-imag_true)**2)) / n_points
@tf.function
def ComplexMAE(y_true, y_pred):
    shape = y_pred.get_shape().as_list()
    if shape[-1]%2:
        raise ValueError(f"nb of imaginary part isn't equal to that of real part")
    num_channels = shape[-1]//2
    n_points = 1
    for n in shape[1:]:
        n_points = n_points*n
    n_points = tf.cast(n_points,dtype=tf.float32)/2
    real_pdt = y_pred[:,:,:,num_channels:]
    imag_pdt = y_pred[:,:,:,:num_channels]
    real_true = y_true[:,:,:,num_channels:]
    imag_true = y_true[:,:,:,:num_channels]
    return tf.reduce_sum(tf.sqrt((real_pdt-real_true)**2+(imag_pdt-imag_true)**2)) / n_points


# Drop version AMU

In [4]:
'''
受限於keras，無法處理孤兒
'''
class AmplitudeMaxout_drop(Layer):
    def __init__(self, num_pieces, name='AMU',**kwargs):
        super().__init__(**kwargs)
        self.__name__ = name
        self.num_pieces = num_pieces    
  
    @tf.function
    def call(self, x, axis=None):
        shape = x.get_shape().as_list()
        if axis is None:
            axis = -1
            shape[0] = -1
        # nb of Re/Im channels
        num_channels = shape[axis]//2
        real_part = x[:,:,:,:num_channels]
        imag_part = x[:,:,:,num_channels:]
        # if # of Re != Im channels
        if shape[axis]%2:
            raise ValueError(f'nb of real/imaginary channel are inequivalent')
        if num_channels%self.num_pieces:
            num_truncated = num_channels%self.num_pieces
            real_part = real_part[:,:,:,:-num_truncated]
            imag_part = imag_part[:,:,:,:-num_truncated]
        # Re/Im channel 每num_piece比較一次，因此最後會有num_units個channel   
        self.num_units = num_channels//self.num_pieces
        # 增加一個dim，最後一個dim為比較大小用，最後會被壓縮，壓縮至倒數第二個channel，因此最後會有num_units個channel
        shape[axis] = self.num_units
        exp_shape = shape + [self.num_pieces]
        # 若輸入的Re/Im channel數量無法被num_pieces整除，需分開處理整除以及剩下的部分
        real_part = tf.reshape(real_part,exp_shape)
        imag_part = tf.reshape(imag_part,exp_shape)
        real_part, imag_part = self.return_AMU(real_part, imag_part, exp_shape)
        return tf.concat([real_part,imag_part],axis=-1)

    def compute_output_shape(self, input_shape):
        shape = list(input_shape)
        shape[-1] = 2*self.num_units
        return tuple(shape)
    
    def get_config(self):
        base_config = super().get_config()
        config = {'units': self.num_units,
                  'pieces': self.num_pieces}
        return dict(list(base_config.items()) + list(config.items()))

    def return_AMU(self, real_part, imag_part ,expand_shape):
        modulus = real_part**2 + imag_part**2
        expand_modulus = tf.reshape(modulus, expand_shape)

        cond = tf.equal(expand_modulus,tf.reduce_max(expand_modulus,axis=-1,keepdims=True))
        real_part = tf.reduce_max(real_part*tf.cast(cond,dtype=tf.float32),axis=-1)
        imag_part = tf.reduce_max(imag_part*tf.cast(cond,dtype=tf.float32),axis=-1)

        return real_part, imag_part


# Final version AMU

In [5]:
'''
受限於keras，無法處理孤兒
'''
class AmplitudeMaxout_inclued(Layer):
    def __init__(self, num_pieces, name='AMU',**kwargs):
        super().__init__(**kwargs)
        self.__name__ = name
        self.num_pieces = num_pieces
        
    @tf.function
    def call(self, x, axis=None):
        shape = x.get_shape().as_list()
        print('AMU',shape)
        if axis is None:
            axis = -1
            shape[0] = -1
        # if # of Re != Im channels
        if shape[axis]%2:
            raise ValueError(f'nb of real/imaginary channel are inequivalent')
        num_channels = shape[-1]//2
        self.num_units = num_channels//self.num_pieces

        if num_channels%self.num_pieces:
            self.num_units += 1
            num_padding = self.num_pieces - num_channels%self.num_pieces
            padding_size = tf.concat([tf.shape(x)[:-1],tf.constant([num_padding])],axis=-1)
            zero_padding = tf.zeros(padding_size)
            
        shape[axis] = self.num_units
        exp_shape = shape + [self.num_pieces]
        real_part = x[:,:,:,:num_channels]
        imag_part = x[:,:,:,num_channels:]
        
        if num_channels%self.num_pieces:
            real_part = tf.concat([real_part,zero_padding], axis=-1)
            imag_part = tf.concat([imag_part,zero_padding], axis=-1)
        real_part = tf.reshape(real_part, exp_shape)
        imag_part = tf.reshape(real_part, exp_shape)
        real_part, imag_part = self.return_AMU(real_part, imag_part, exp_shape)
        return tf.concat([real_part,imag_part],axis=-1)         

    def compute_output_shape(self, input_shape):
        shape = list(input_shape)
        shape[-1] = 2*self.num_units
        return tuple(shape)
    
    def get_config(self):
        base_config = super().get_config()
        config = {'units': self.num_units,
                  'pieces': self.num_pieces}
        return dict(list(base_config.items()) + list(config.items()))
    def return_AMU(self, real_part, imag_part ,expand_shape):

        modulus = real_part**2 + imag_part**2
        expand_modulus = tf.reshape(modulus, expand_shape)    
        cond = tf.equal(expand_modulus,tf.reduce_max(expand_modulus,axis=-1,keepdims=True))
        real_part = tf.reduce_max(real_part*tf.cast(cond,dtype=tf.float32),axis=-1)
        imag_part = tf.reduce_max(imag_part*tf.cast(cond,dtype=tf.float32),axis=-1)
        
        return real_part, imag_part

In [ ]:
tf.keras.backend.clear_session()
x_train = np.random.rand(61,64,64,32)
y_train = np.random.rand(61,64,64,2)
input_shapes = x_train.shape[1:]

InputTensor = keras.Input(shape=input_shapes)
conv1 = complexnn.conv.ComplexConv2D(64,(3,3),padding='same')(InputTensor)
amp1 = AmplitudeMaxout_inclued(4)(conv1)
conv2 = complexnn.conv.ComplexConv2D(16,(3,3),padding='same')(amp1)
amp2 = AmplitudeMaxout_inclued(4)(conv2)
incep = InceptLayer(filter1=4,
                    filter2=4,
                    filter3=4,
                    filter4=4,
                    kernel1=(3,5),
                    kernel2=(3,3),
                    kernel3=(5,3),
                    kernel4=(3,5),
                    num_units=4)(amp2)
Output = complexnn.conv.ComplexConv2D(1,(1,1),padding='same')(incep)

model = keras.Model(inputs=InputTensor,outputs=Output)


# model.add(AmplitudeMaxout(8))
# model.add(AmplitudeMaxout_piece(8))
# model.add(complexnn.conv.ComplexConv2D(64,(3,3),padding='same'))
# model.add(AmplitudeMaxout(4))
# model.add(InceptLayer(filter1=4, filter2=4, filter3=4, filter4=4, kernel1=(3,5), kernel2=(3,3), kernel3=(5,3), kernel4=(3,5)))
# model.add(complexnn.conv.ComplexConv2D(1,(1,1),padding='same'))
model.compile(optimizer=keras.optimizers.RMSprop(),loss=ComplexMSE)
model.build
# model.predict(x_train)
model.fit(x_train,y_train,epochs=2)


AMU [None, 64, 64, 128]
AMU [None, 64, 64, 32]
inputshape (None, 64, 64, 8)
AMU [None, 64, 64, 8]
real1 (None, 64, 64, 1) image1 (None, 64, 64, 1)
real2 (None, 64, 64, 1) image2 (None, 64, 64, 1)
real3 (None, 64, 64, 1) image3 (None, 64, 64, 1)
real4 (None, 64, 64, 1) image4 (None, 64, 64, 1)
what (None, 64, 64, 8)
last_dim1 2
computeshape [None, 64, 64, 8]
Epoch 1/2


In [8]:
tf.keras.backend.clear_session()
input_shapes = x_train.shape[1:]

InputTensor = keras.Input(shape=input_shapes)
conv1 = complexnn.conv.ComplexConv2D(64,(3,3),padding='same',activation=complexnn.activation.CReLU)(InputTensor)
amp1 = complexnn.AMU(4)(conv1)
conv2 = complexnn.conv.ComplexConv2D(16,(3,3),padding='same',activation=complexnn.activation.CReLU)(amp1)
amp2 = complexnn.AMU(4)(conv2)
Output = complexnn.conv.ComplexConv2D(1,(1,1),padding='same',activation=complexnn.activation.zReLU)(amp2)
model = keras.Model(inputs=InputTensor,outputs=Output)
model.compile(optimizer=keras.optimizers.RMSprop(),loss=ComplexMSE)
model.build

AttributeError: 'tuple' object has no attribute 'layer'

In [334]:
def decorator_AMU(func):
    def run_AMU(y):
        # 再透過first讀取y值也就是2,3*2+執行func獲得的值
        value = func
        return 3*y + value
    return run_AMU
def AMU_here(x):
    # decorator_AMU讀取AMU_here(3)然後獲得first=run_AMU
    return 5*x
first = decorator_AMU(AMU_here(3))
first(2)

21

In [135]:
model.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 64, 64, 4)         0         
_________________________________________________________________
complex_conv2d_1 (ComplexCon (None, 64, 64, 128)       2432      
_________________________________________________________________
amplitude_maxout_inclued_1 ( (None, 64, 64, 32)        0         
_________________________________________________________________
complex_conv2d_2 (ComplexCon (None, 64, 64, 32)        4640      
_________________________________________________________________
amplitude_maxout_inclued_2 ( (None, 64, 64, 8)         0         
_________________________________________________________________
incept_layer_1 (InceptLayer) (None, 64, 64, 8)         1760      
_________________________________________________________________
amplitude_maxout_inclued_4 ( (None, 64, 64, 4)         0   

In [90]:
maxvalue = tf.reduce_max(x,axis=-1,keepdims=True)
print(tf.equal(x,maxvalue))

tf.Tensor(
[[[[ True False]
   [ True False]
   [False  True]
   ...
   [ True False]
   [False  True]
   [False  True]]

  [[False  True]
   [False  True]
   [False  True]
   ...
   [ True False]
   [False  True]
   [False  True]]

  [[ True False]
   [False  True]
   [False  True]
   ...
   [False  True]
   [False  True]
   [False  True]]

  ...

  [[ True False]
   [False  True]
   [False  True]
   ...
   [False  True]
   [False  True]
   [ True False]]

  [[False  True]
   [ True False]
   [ True False]
   ...
   [False  True]
   [ True False]
   [False  True]]

  [[ True False]
   [ True False]
   [False  True]
   ...
   [ True False]
   [False  True]
   [False  True]]]], shape=(1, 64, 64, 2), dtype=bool)


In [111]:
tf.reduce_max(x*tf.cast(cond,dtype=tf.float32),axis=-1,keepdims=True)

<tf.Tensor: shape=(1, 64, 64, 1), dtype=float32, numpy=
array([[[[ 0.693821  ],
         [ 1.1007732 ],
         [ 0.11753968],
         ...,
         [ 1.4398488 ],
         [-0.        ],
         [-0.        ]],

        [[-0.        ],
         [-0.        ],
         [ 2.690458  ],
         ...,
         [ 0.30257797],
         [ 0.21762794],
         [ 1.1404315 ]],

        [[-0.        ],
         [-0.        ],
         [ 0.9346983 ],
         ...,
         [ 0.28814462],
         [ 0.568626  ],
         [ 0.17965187]],

        ...,

        [[ 1.1098131 ],
         [ 0.14743602],
         [ 0.26171896],
         ...,
         [-0.        ],
         [ 0.47875816],
         [ 2.019243  ]],

        [[ 1.631837  ],
         [-0.        ],
         [ 0.3161689 ],
         ...,
         [-0.        ],
         [-0.        ],
         [ 0.48682392]],

        [[-0.        ],
         [ 2.8689127 ],
         [-0.        ],
         ...,
         [ 1.45738   ],
         [ 1.2759118

In [97]:
x

TensorShape([1, 64, 64, 2])

由於keras.backend.conv2d與tf.nn.conv2d使用的weights random seed不同，因此算出來的值有些微差異

In [24]:
def test_conv_keras(x,w,b):
    x_shape = list(x.shape)
    x_dim = x_shape[-1]//2
    xre = x[:,:,:,:x_dim]
    xim = x[:,:,:,x_dim:]
    
    w_shape = list(w.shape)
    w_dim = w_shape[-1]//2
    new_shape = [w_shape[0],w_shape[1],x_dim,w_dim]
    wre = w[:,:,:,:w_dim].reshape(new_shape)
    wim = w[:,:,:,w_dim:].reshape(new_shape)
    
    xre = tf.convert_to_tensor(xre,dtype=tf.float32)
    xim = tf.convert_to_tensor(xim,dtype=tf.float32)
    crr = tf.keras.backend.conv2d(xre, wre, strides=1, padding='same')

    cii = tf.keras.backend.conv2d(xim, wim, strides=1, padding='same')

    cri = tf.keras.backend.conv2d(xre, wim, strides=1, padding='same')

    cir = tf.keras.backend.conv2d(xim, wre, strides=1, padding='same')
    return tf.nn.bias_add(np.concatenate([crr-cii,cri+cir],axis=-1),b)
wb = model.get_weights()
w = wb[0]
b = wb[1]
y1 = test_conv_keras(x_train,w,b)
print(y1)

tf.Tensor(
[[[[ 6.25617895e-03  2.03109175e-01 -9.14481133e-02 ...  2.63803098e-02
    -6.93420172e-02  9.75780636e-02]
   [-1.46586150e-01  1.60859004e-01  5.28705269e-02 ... -5.41173015e-03
    -7.07495138e-02  1.96250543e-01]
   [-2.34843209e-01 -5.58444820e-02  1.59698859e-01 ... -4.74821497e-03
    -3.67917530e-02  7.05122650e-02]
   ...
   [-2.77378738e-01 -1.74655318e-01  2.00963974e-01 ...  2.94435266e-02
     9.70133469e-02  1.52889937e-01]
   [-2.67395347e-01  9.19701383e-02  1.07647166e-01 ...  2.35644262e-02
     2.21778918e-02 -5.10653891e-02]
   [-1.02950752e-01 -3.92761678e-02  3.09133887e-01 ...  1.42906442e-01
    -9.74455383e-03 -2.34643240e-02]]

  [[-1.21079542e-01  2.15288892e-01 -2.36096773e-02 ... -1.60411447e-01
     8.65850449e-02 -1.42354429e-01]
   [-7.88681805e-02  8.57565552e-02  1.04116917e-01 ... -2.57048127e-03
     4.75392397e-03  6.07592426e-02]
   [-2.03685492e-01  7.83405676e-02  1.75316676e-01 ...  1.04833789e-01
    -3.58269475e-02  7.91847706e-02]

In [30]:
AMU2 = AmplitudeMaxout_piece(8)
AMU2(y1)

<tf.Tensor 'amplitude_maxout_piece_5/StatefulPartitionedCall:0' shape=(None, 64, 64, 16) dtype=float32>

In [33]:
print(tf.shape(AMU2(y1)))

Tensor("Shape:0", shape=(4,), dtype=int32)


In [18]:
wb = model.get_weights()
w = wb[0]
b = wb[1]
# filter shape = [h,w,in,out]
wre = w[:,:,:,:16].reshape(3,3,2,16)
wim = w[:,:,:,16:].reshape(3,3,2,16)
xre = x_train[:,:,:,:2].reshape(1,5,5,2)
xim = x_train[:,:,:,2:].reshape(1,5,5,2)
bre = b[:16]
bim = b[16:]

crr = tf.nn.conv2d(xre, wre, strides=1, padding='SAME')

cii = tf.nn.conv2d(xim, wim, strides=1, padding='SAME')

cri = tf.nn.conv2d(xre, wim, strides=1, padding='SAME')

cir = tf.nn.conv2d(xim, wre, strides=1, padding='SAME')

c = np.concatenate([crr-cii,cri+cir],axis=-1)
#c = tf.nn.bias_add(c,b)
print(c)

ValueError: cannot reshape array of size 2016 into shape (3,3,2,16)

In [198]:
xre = tf.convert_to_tensor(xre,dtype=tf.float32)
xim = tf.convert_to_tensor(xim,dtype=tf.float32)
crr = tf.keras.backend.conv2d(xre, wre, strides=1, padding='same')

cii = tf.keras.backend.conv2d(xim, wim, strides=1, padding='same')

cri = tf.keras.backend.conv2d(xre, wim, strides=1, padding='same')

cir = tf.keras.backend.conv2d(xim, wre, strides=1, padding='same')

c = np.concatenate([crr-cii,cri+cir],axis=-1)
#c = tf.nn.bias_add(c,b)
print(c)

[[[[ 9.26236734e-02  5.56264371e-02 -3.75130475e-02  2.75679827e-02
    -4.98209856e-02  7.03494549e-02 -1.29094180e-02 -5.38495891e-02
    -2.95863301e-03 -2.28326023e-01  1.42695501e-01 -5.05730547e-02
     1.60340462e-02  4.10894305e-02 -8.17370117e-02 -2.51134753e-01
     1.45940050e-01 -9.82771814e-02 -9.67776701e-02  5.10786921e-02
    -4.05824669e-02  5.12620397e-02  1.02818340e-01 -3.54281850e-02
    -1.24067448e-01  5.89118078e-02 -2.00985894e-02  7.68649951e-02
     9.95618254e-02  3.98461372e-02 -6.46891445e-02  1.47891209e-01]
   [ 1.72541916e-01  1.21158153e-01  6.05145022e-02 -1.45997936e-02
    -1.79254726e-01  2.49485806e-01  1.38307169e-01 -1.98406398e-01
     1.40651152e-01 -4.91308235e-02  1.10937953e-01 -1.21211484e-02
     1.32695660e-02  1.39820188e-01 -9.73012298e-02 -2.25816935e-01
     1.61593407e-01  5.60295954e-02 -2.46427551e-01  6.88612014e-02
    -5.29930256e-02  1.98219627e-01 -2.12729394e-01  2.34849542e-01
    -1.24716446e-01  4.21086475e-02  6.58788830

In [8]:
x = tf.random.normal([1,5,5,4],dtype=tf.float32)
x = np.random.randn(1,5,5,4)
input_img = keras.layers.Input(shape=(5,5,4))
output = complexnn.conv.ComplexConv2D(16,(3,3),activation='relu',padding='same')(input_img)
model = keras.Model(inputs=input_img, outputs=output)
print(model.predict(x))

Tensor("complex_conv2d_5/convolution:0", shape=(None, 5, 5, 32), dtype=float32)
[[[[0.         0.3711054  0.         0.         0.09828413 0.0510309
    0.04386184 0.         0.         0.37805748 0.         0.80737066
    0.         0.         0.         0.4755184  0.08107495 0.27993476
    0.04829627 0.1474599  0.198298   0.00270782 0.         0.0485894
    0.0311655  0.13569205 0.23479684 0.         0.         0.23957776
    0.         0.05887367]
   [0.32442778 0.         0.20647484 0.12051331 0.         0.
    0.         0.         0.         0.         0.         0.01839001
    0.         0.         0.07516283 0.         0.12995952 0.29833636
    0.         0.         0.         0.45695427 0.         0.6990275
    0.         0.         0.55132717 0.         0.         0.05588689
    0.         0.        ]
   [0.23193896 0.         0.03455663 0.3761718  0.20444919 0.
    0.3445665  0.         0.         0.         0.08437333 0.
    0.28718892 0.         0.         0.         0.144

In [22]:
test_conv_keras(x_train,w,b)

<tf.Tensor: shape=(61, 64, 64, 128), dtype=float32, numpy=
array([[[[ 6.25617895e-03,  2.03109175e-01, -9.14481133e-02, ...,
           2.63803098e-02, -6.93420172e-02,  9.75780636e-02],
         [-1.46586150e-01,  1.60859004e-01,  5.28705269e-02, ...,
          -5.41173015e-03, -7.07495138e-02,  1.96250543e-01],
         [-2.34843209e-01, -5.58444820e-02,  1.59698859e-01, ...,
          -4.74821497e-03, -3.67917530e-02,  7.05122650e-02],
         ...,
         [-2.77378738e-01, -1.74655318e-01,  2.00963974e-01, ...,
           2.94435266e-02,  9.70133469e-02,  1.52889937e-01],
         [-2.67395347e-01,  9.19701383e-02,  1.07647166e-01, ...,
           2.35644262e-02,  2.21778918e-02, -5.10653891e-02],
         [-1.02950752e-01, -3.92761678e-02,  3.09133887e-01, ...,
           1.42906442e-01, -9.74455383e-03, -2.34643240e-02]],

        [[-1.21079542e-01,  2.15288892e-01, -2.36096773e-02, ...,
          -1.60411447e-01,  8.65850449e-02, -1.42354429e-01],
         [-7.88681805e-02,  8

In [236]:
wb = model.get_weights()
weight = {}
for i,value in enumerate(wb):
    weight[f"w{i}"] = value 

In [238]:
tf_upgrade_v2 -h

NameError: name 'tf_upgrade_v2' is not defined

In [327]:
class Parent():
    def __init__(self,**kwargs):
        if 'filter1' in kwargs.keys():
            self.filter1 = kwargs['filter1']
class Child(Parent):
    def __init__(self,**kwargs):
        super().__init__(**kwargs)
        if 'filter2' in kwargs.keys():
            self.filter2 = kwargs['filter2']
    def call(self):
        print(self.filter1)
value = Child(filter1=3)
value.call()

AttributeError: 'Child' object has no attribute '__method__'

In [117]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
complex_conv2d_1 (ComplexCon (None, 7, 7, 32)          608       
_________________________________________________________________
amplitude_maxout_1 (Amplitud (None, 7, 7, 8)           0         
_________________________________________________________________
incept_layer_1 (InceptLayer) (None, 7, 7, 8)           1760      
_________________________________________________________________
complex_conv2d_6 (ComplexCon (None, 7, 7, 2)           10        
Total params: 2,378
Trainable params: 2,378
Non-trainable params: 0
_________________________________________________________________


In [466]:
testx = np.random.randn(1,5,5,3)
testw = np.random.randn(5,5,3,1)
print(tf.nn.conv2d(testx,testw,strides=1,padding='SAME'))

tf.Tensor(
[[[[-8.75485408]
   [-5.39883325]
   [-1.46161579]
   [ 2.1202254 ]
   [-5.8274704 ]]

  [[-8.60364303]
   [-3.06373836]
   [-1.70510369]
   [ 3.52396714]
   [16.05740424]]

  [[ 4.894179  ]
   [ 0.31499733]
   [ 9.01325559]
   [ 9.26489865]
   [-3.57912787]]

  [[-0.35757337]
   [ 1.26762377]
   [ 9.59263609]
   [-5.17463597]
   [ 9.14126604]]

  [[ 0.38725434]
   [-0.14484194]
   [ 2.11834608]
   [-3.72945311]
   [-5.90346486]]]], shape=(1, 5, 5, 1), dtype=float64)


In [131]:
model.weights[1].shape

TensorShape([32])

In [353]:
class TestLayer(Layer):
    def __init__(self,**kwargs):
        super().__init__(**kwargs)
        for key,item in kwargs.items():
            print(key,item)
TestLayer()

In [46]:
wre.shape

(1, 3, 3, 1)

In [44]:
w = model.get_weights()
w = w[0]
print(w[:,:,:,0].shape)

(3, 3, 1)


In [29]:
model.predict(x_train)

array([[[[0.        , 0.11565769],
         [0.        , 0.        ],
         [0.        , 0.        ],
         [0.        , 0.06815258],
         [0.        , 0.        ]],

        [[0.        , 0.27141032],
         [0.        , 0.        ],
         [0.        , 0.09650331],
         [0.07783239, 0.12203552],
         [0.        , 0.        ]],

        [[0.04767413, 0.        ],
         [0.00180166, 0.04797432],
         [0.        , 0.11543889],
         [0.        , 0.20652491],
         [0.        , 0.        ]],

        [[0.03345764, 0.34430975],
         [0.        , 0.07456253],
         [0.05984167, 0.        ],
         [0.        , 0.11067256],
         [0.        , 0.        ]],

        [[0.0190777 , 0.13882321],
         [0.09074385, 0.        ],
         [0.16623655, 0.        ],
         [0.        , 0.        ],
         [0.        , 0.        ]]]], dtype=float32)

In [5]:
x = tf.convert_to_tensor(x_train)
x = tf.cast(x,dtype=tf.float32)
Test = complexnn.conv.ComplexConv2D(32,(3,3),activation='relu',padding='same',input_shape=(5,5,2))
Test(x)

kernel_shape is: (3, 3, 1, 32)
in add weight (3, 3, 1, 32)
filter size: 32
kernel size: (3, 3, 1, 64)


<tf.Tensor 'complex_conv2d_3/Relu:0' shape=(1, 5, 5, 64) dtype=float32>

In [8]:
Test.kernel

<tf.Variable 'complex_conv2d_4/kernel:0' shape=(3, 3, 1, 64) dtype=float32, numpy=
array([[[[-3.44312191e-02,  3.81694324e-02,  8.87103453e-02,
          -4.40639667e-02,  1.50182424e-03,  1.29781544e-01,
           2.75353249e-02,  1.86049759e-01, -1.72066778e-01,
          -4.25088480e-02, -5.29412366e-03,  1.46391660e-01,
          -5.63969351e-02, -2.28644609e-01,  7.53736123e-02,
          -1.73939124e-01, -1.01491258e-01, -1.04216710e-01,
          -3.86741534e-02,  9.33780298e-02,  1.14866361e-01,
           1.21312169e-02, -8.76665190e-02,  1.44240633e-01,
          -4.99256179e-02, -1.70480832e-02,  8.37967694e-02,
           3.42274643e-02,  7.95376673e-02,  1.48694709e-01,
           2.68359911e-02,  3.63685773e-03,  7.94785172e-02,
          -5.30043654e-02,  1.33236917e-02, -1.15072258e-01,
          -9.77580249e-02,  3.45963836e-02, -8.21061134e-02,
           2.39926621e-01, -5.45983762e-02, -3.48874554e-02,
           1.09665409e-01, -5.49463704e-02,  1.08670099e-02,
  

In [9]:
Test.filters

32

In [10]:
Test.kernel_shape

(3, 3, 1, 32)

In [6]:
Test.get_config()

{'name': 'complex_conv2d_3',
 'trainable': True,
 'batch_input_shape': (None, 5, 5, 2),
 'dtype': 'float32',
 'filters': 32,
 'kernel_size': (3, 3),
 'strides': (1, 1),
 'padding': 'same',
 'data_format': 'channels_last',
 'dilation_rate': (1, 1),
 'activation': 'relu',
 'use_bias': True,
 'normalize_weight': False,
 'kernel_initializer': 'complex',
 'bias_initializer': {'class_name': 'Zeros', 'config': {}},
 'gamma_diag_initializer': 'sqrt_init',
 'gamma_off_initializer': {'class_name': 'Zeros', 'config': {}},
 'kernel_regularizer': None,
 'bias_regularizer': None,
 'gamma_diag_regularizer': None,
 'gamma_off_regularizer': None,
 'activity_regularizer': None,
 'kernel_constraint': None,
 'bias_constraint': None,
 'gamma_diag_constraint': None,
 'gamma_off_constraint': None,
 'init_criterion': 'he',
 'spectral_parametrization': False,
 'transposed': False}

In [7]:
Test.kernel[:,:,:,:Test.filters]

<tf.Tensor: shape=(3, 3, 1, 32), dtype=float32, numpy=
array([[[[-3.44312191e-02,  3.81694324e-02,  8.87103453e-02,
          -4.40639667e-02,  1.50182424e-03,  1.29781544e-01,
           2.75353249e-02,  1.86049759e-01, -1.72066778e-01,
          -4.25088480e-02, -5.29412366e-03,  1.46391660e-01,
          -5.63969351e-02, -2.28644609e-01,  7.53736123e-02,
          -1.73939124e-01, -1.01491258e-01, -1.04216710e-01,
          -3.86741534e-02,  9.33780298e-02,  1.14866361e-01,
           1.21312169e-02, -8.76665190e-02,  1.44240633e-01,
          -4.99256179e-02, -1.70480832e-02,  8.37967694e-02,
           3.42274643e-02,  7.95376673e-02,  1.48694709e-01,
           2.68359911e-02,  3.63685773e-03]],

        [[ 9.59886834e-02, -1.23542091e-02, -1.27712876e-01,
          -1.67284906e-02,  1.09242812e-01,  3.36131811e-01,
          -6.56206235e-02, -1.47943823e-02,  8.15441534e-02,
          -1.53653204e-01,  6.71673473e-03,  9.88173038e-02,
           3.31760310e-02, -1.00749761e-01, 

In [13]:
g = {
    'a':3,
    'b':4
}
for uu,kk = g.items()

dict_items([('a', 3), ('b', 4)])